# Graph Neural Networks

---
**Lernziele**
* Sie verstehen, wie Moleküle für den Computer als Graph dargestellt werden können. 
* Sie verstehen, wie Graph Neural Networks funktionieren.
* Sie können ein Graph Neural Network als Pytorch Klasse schreiben.
---

Graph Neural Networks sind noch eine relativ neue Methode. Intuitiv können wir sehen, dass sich Moleküle sehr gut als (mathematischen) Graph darstellen lassen. Dabei entsprechen die Bindungen im Molekül den Kanten (edges) des Graphs und die Atome  den Knoten (nodes).<br> Für den Computer sind jedoch Graphen nicht so leicht zu lesen wie zum Beispiel ein SMILES string. Wir stellen die Moleküle als zwei Matrizen dar. Zum einen die Adjacency-Matrix, die die Atomverknüpfung (also die Bindungen) darstellt. Dazu kommen dann noch die Feature-Matrix. Für das Machine Learning One-Hot koidieren wir hier die Atome.<br><br>
<center>
<img src="https://www.researchgate.net/profile/Jorge_Galvez2/publication/236018587/figure/fig1/AS:299800013623305@1448489301609/The-chemical-graph-and-adjacency-matrix-of-the-isopentane.png" style="width: 600px;">
</center>
<h8><center>Galvez et. al. 2010</center></h8><br><br>
Wir nehmen nochmal die Daten aus der Tox21 Challenge. 

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem.rdmolops import GetAdjacencyMatrix
import numpy as np
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils import data
import math
from sklearn.metrics import roc_auc_score
%run ../utils/onehotencoder.py
np.set_printoptions(linewidth=300)

In [ ]:
# Laden der Daten
data_tox = pd.read_csv("../data/toxicity/sr-mmp.tab", sep = "\t")
data_tox = data_tox.iloc[:,1:] #alle Spalten bis auf die erste (index 0) werden ausgewählt
data_tox.columns = ["smiles", "activity"]
data_tox.head()

## Adjacency Matrix und One-Hot Encoded Feature Matrix
Mit den Smiles können wir diesmal leider  nicht so viel mit anfangen. Aber zum Glück gibt es in RDKit direkt eine Funktion, um die Adjacency Matrix zu erstellen. Diese haben wir oben schon importiert.


In [ ]:
mols = [Chem.MolFromSmiles(x) for x in data_tox['smiles']]
A = [GetAdjacencyMatrix(x) for x in mols]
print(A[1])

Um die Atome zu One-Hot Encoden benutzen wir die vorgeschriebene `onehotencode()` Funktion.
Ähnlich wie bei RNNs Smilestoken kodiert werden, wird hier nur der Atomtype eines jedes Atoms erfasst. Diese Atome werden dann in one-hot kodiert.
Pro Moleküle werden diese Vektoren untereinander zu einer Matrix kombiniert.

In [ ]:
feat = onehotencode(mols)
feat[1]

Oben ist zusehen, wie eine Feature-Matrix für ein Molekül aussieht.
Wenn wir uns die `.shape` ansehen, können wir erkennen, dass unser Molekül aus `29` Atomen besteht. Insgesamt immer Datensatz gibt es `25` Atomtypen. Die Anzahl Spalten (Anzahl der Features) muss für alle Moleküle gleich sein, sonst können wir die Moleküle nicht durch das Netzwerk führen.

In [ ]:
feat[1].shape

Vielleicht ist ihnen aufgefallen, dass auf der Diagonalen der Adjacency Matrix noch Nullen stehen. In einer Graph Convolution sollen aber nicht nur die Features der benachbarten, sondern auch des zentralen Atoms mit in die Rechnung genommen werden. Dafür werden Einsen auf der Diagonalen der Adjacency-Matrix benötigt. 
Die Funktion `np.fill_diagonal(matrix, value)` erlaubt es Ihnen, die Werte der Diagonalen einer Matrix zu ändern. 

In [ ]:
for matrix in A:
    np.fill_diagonal(matrix, 1)
print(A[1])

## Graph Convolution
Jetzt wollen wir die Informationen der Knoten an die Nachbarknoten entlang der Kanten weiterzugeben, was als (Graph) Convolution bezeichnet wird. Hierfür werden während des forward pass diese mathematischen Operationen des ausgeführt: $$\hat{X} = \hat{D}^{-1}\hat{A}XW$$
Hier ist $\hat{A}$ unsere Adjacency Matrix, mit Einsen auf der Diagonalen. $X$ ist die Feature-Matrix und $W$ sind die Weights, die pytorch initialisiert. Es fehlt noch $D$, die Degree Matrix. Diese Matrix enthält die Anzahl der Verbindungen, die jedes Atom im Molekül hat. Diese Werte werden in der Degree-Matrix auf der Diagonalen platziert. Die Anzahl der Verbindung jedes Moleküls lässt sich leicht aus der Adjacency-Matrix berechnen. Dafür müssen Sie die Summe der Reihe oder Spalten der$\hat{A}$ Matrix berechnen. Diese Summe gibt die Degrees eines Atom an. Um genau zu sein, ist es der Degree + 1 des Atoms, da Sie die Diagonale der Adjacency-Matrix bereits mit Einsen gefüllt haben.
Um die Degree-Matrix zu erstellen, müssen Sie die Summen der Reihen in jeder Adjacency-Matrix berechnen und dieses auf die Diagonale einer neuen Matrix platzieren.


Dieser Prozess ist relativ simpel mit `numpy`. `np.sum(A[i], axis=0)` berechnet die Summe pro Spalte und `np.diag()` erstellt aus einem 1D-Array eine Matrix mit den Werten des 1D Arrays auf der Diagonalen. Sie können die beiden Funktion in Kombination benutzen, um die Degree-Matrizen zu erstellen.

In [ ]:
D =[]
for matrix in A:
    D.append(np.diag(np.sum(matrix, axis=1)))
print(D[1])

Bevor wir anfangen ein Netzwerk zu bauen und es mit unseren Daten zu füttern, können wir noch einen Schritt durchführen, um einiges an Rechenaufwand zu sparen, denn $\hat{D}^{-1}\hat{A}$ kann schon vor dem Trainieren das Netzwerk berechnet werden. Dadurch kann Zeit gespart werden, da dieser Schritt nicht ständig im Netzwerk wiederholt werden muss.

`np.linalg.matrix_power` wird benötigt, um das Invers der Matrix `D` zu berechnen. Ansonsten ist dieser Schritt eine einfache Matrixmultiplikation.

In [ ]:
DA = []
for i in range(len(D)):
    DA.append(np.matmul(np.linalg.matrix_power(D[i], -1),A[i]))
DA[0]

Wir haben also die Liste der Adjacency Matrizen `DA`. Diese enthält die Informationen über die Struktur des Moleküls. Die Liste `feat` enthält die Features, also die Informationen zu den einzelnen Knoten. Und als Letztes erstellen wir noch eine `labels` Liste. Diese enthält die Information, die wir vorhersagen wollen (toxisch vs. nicht toxisch).

In [ ]:
DA = [torch.tensor(x,dtype=torch.float32) for x in DA] # Konvertieren der Arrays zu Tensoren
feat = [torch.tensor(x,dtype=torch.float32) for x in feat] # Konvertieren der Arrays zu Tensoren
labels = [torch.tensor([x], dtype=torch.float32) for x in data_tox['activity']]

## Graph Convolution Layer
Wir wollen uns für die Graph Convolution die Struktur von PyTorch zunutze machen. Wie schon letzte Woche verwenden wir dafür Klassen.
Letzte Woche haben wir eine Klasse benutzt um aus verschiedene Layern (z.B. `nn.Linear()`)ein Netzwerk zu erstellen. 
Diese Layers sind in pyTorch schon vorgeschrieben, aber auch wir können unsere eigenen Layer erstellen. Auch hierfür brauchen wir wieder die PyTorch Klasse.
Unser Ziel ist es, so etwas wie `nn.Linear()` zu programmieren, sodass wir unser Repertoire an Hidden Layers (Linear, RNN, GRU, Dropout, ...) um eine Graph Convolution Layer erweitern.<br> Auch dafür können wir die`nn.Module` Klasse als Basis für unsere Graph Convolution verwenden. Wir fangen mit dem minimalen Grundgerüst an:
```python
class GraphConvolution(nn.Module):
    pass
```
Damit wir richtig auf die Funktionen der übergeordneten `nn.Module` Klasse zugreifen können, müssen wir diese mit `super().__init__()` initialisieren. Unsere Convolutional Layer muss natürlich wissen, wie groß der Input ist, und wie groß der Output werden soll. Außerdem müssen wir hier selbst die Weights und Biases initialisieren.
```python
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_featuers = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.bias = nn.Parameter(torch.FloatTensor(out_features))
```
Dann können wir auch schon die `forward()` Funktion programmieren. Hier passiert die eigentliche Convolution, bei der die Knoten mit den Features `x` durch Matrixmultiplikation mit der Adjacency Matrix `adj` über ihre Nachbarknoten lernen. Indem wir noch die lernbaren `weights` dazwischenschalten, können wir das Ganze später optimieren.
```python
    def forward(self, x, adj):
        support = torch.mm(x, self.weight)
        output = torch.mm(adj, support)
        return output + self.bias
```
Damit würde unsere `GraphConvolution` Klasse auch schon funktionieren. Wir initialisieren die Weights aber noch etwas optimiert mit der Funktion `reset_parameters()`. Und mit der Funktion `__repr__()` können wir uns die Layer noch richtig anzeigen lassen. Unsere finale Klasse sieht dann so aus:

In [ ]:
class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        self.bias = nn.Parameter(torch.FloatTensor(out_features))
        self.reset_parameters()
    
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(0, stdv)
        self.bias.data.uniform_(-stdv, stdv)
        
    def forward(self, x, adj):
        support = torch.mm(x, self.weight)
        output = torch.mm(adj, support)
        return output + self.bias
    
    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + 'in_features=' + str(self.in_features) + ', ' \
               + 'out_features=' + str(self.out_features) + ')'

Wir können auch überprüfen, ob unsere Graph Convolution Layer auch schon funktioniert.
Zunächst speichern wir eine Feature-Matrix und eine Adjacency Matrix als Beispiel.

In [ ]:
feat_beispiel = feat[1]
adj_beispiel = DA[1]
print('Features:', feat_beispiel.shape)
print('DA:', adj_beispiel.shape)

Wir erstellen nun eine GraphConvolution. Beachten Sie, dass die Inputgröße, der ersten Layer, der Größe des Featurevektors (Anzahl der Spalten) (`25`) entspricht. Also genauso wie bei einer `nn.Linear` Layer.

In [ ]:
conv = GraphConvolution(25, 100)
conv

Wir können nun das Beispiel durch die Convolution führen. Sie werden sehen, dass sich die Dimension der Features auf 100 vergrößert haben wird.

In [ ]:
output = conv(feat_beispiel, adj_beispiel)
print('\nOutput:', output.size())

## Graph Neural Network
Um jetzt ein Netz zu bauen, können wir wieder die `nn.Module` Basisklasse verwenden. 

Letzte Woche haben wir, unseren eigenen Autoencoder zusammen gestellt. Diese Woche benutzen wir diese Art von Klasse, um mehrere Graph Convolutions miteinander zu verbinden. Wichtig ist aber auch zu beachten, dass wir wie bei regulären Convolution Neural Networks nicht nur Convolutional Layers benutzen können. Im CNN selber müssen wir am Ende den Tensor flatten. Dadurch erhalten wir einen Vektor, der dann durch eine linear Layer geführt werden wird. Das Gleiche gilt auch für Graph Convolutions. Um den Output dieser Graph Convolutions in eine linear Layer zu führen, können wir aber nicht `Flatten` benutzen, sondern wir berechnen einfach den Mittelwert über alle Spalten.

Das macht die Funktion `aggregate()`.

Sie können im Netzwerk erkennen, dass wir zwei Graph Convolution Layers benutzen, und dann eine linear Layer.
Im `forward` Pass wird erst der Input `(x, adj)` durch die erste Graph Layer geführt, dann folgt eine ReLU Funktion. Dann wiederholt sich das selber Spiel für die zweite Convolution. 
Nun wenden wir die `aggregate` Funktion an. Diese berechnet jetzt den Mittelwert eines jeden Features. Der Output dieser Layer hat also in dem Beispiel die Größe `[1,100]`.
Als Letztes benutzen wir die linear Layer, um die Vorhersage zu machen.

In [ ]:
class GraphNN(nn.Module):
    def __init__(self):#in_features, out_features, size_labels):
        super().__init__()
        self.conv1 = GraphConvolution(25, 100)
        self.conv2 = GraphConvolution(100, 100)
        self.lin = nn.Linear(100, 1)
        
    def aggregate(self, convoluted_graph): # we use mean aggregation, max or min could also be used as hyperparameter
        return torch.mean(convoluted_graph, dim=0, keepdim=True)
        
    def forward(self, x, adj):
        x = self.conv1(x, adj)
        x = F.relu(x)
        x = self.conv2(x, adj)
        x = F.relu(x)
        x = self.aggregate(x)
        x = self.lin(x)
        return x

Nun werden wir noch schnell den Datensatz in Training und Testset teilen. Hierfür nehmen wir einfach  die ersten 1800 Moleküle als Trainingsset und den Rest als Testset. Wichtig ist, dass wir in diesem Beispiel keine Minibtaches benutzen. Das liegt daran, dass Featurematrizen der einzelnen Moleküle unterschiedlich groß sind, bzw. unterschiedliche viele Reihen haben. Das heißt wir können sie nicht ohne weiteres, wie zum Beispiel beim CNN als 3D Tensor speichern. Es gibt zwar Möglichkeiten dieses Problem zu lösen, diese sind aber nicht relevant für das Notebook. 

In [ ]:
train_feat = feat[:1800]
train_DA = DA[:1800]
train_labels = labels[:1800]


test_feat = feat[1800:]
test_DA = DA[1800:]
test_labels = labels[1800:]

In [ ]:
gnn = GraphNN()
loss_funktion= nn.BCEWithLogitsLoss()
optimizer=optim.Adam(gnn.parameters(), lr =0.01)

In [ ]:
EPOCHS = 20

for i in range(EPOCHS):
    loss_list_train = []
    acc_list_train= []
    gnn.train()
    for k in range(len(train_feat)):
        optimizer.zero_grad()
    
        output=gnn(train_feat[k], train_DA[k]).flatten()

        loss=loss_funktion(output,train_labels[k])
        loss.backward()
        loss_list_train.append(loss.item())
        optimizer.step()

        acc_list_train.append(np.sum((torch.round(torch.sigmoid(output)) == train_labels[k]).detach().numpy()))
    loss_list_test = []
    acc_list_test= []
    gnn.eval()
    for k in range(len(test_feat)):
    
        output=gnn(test_feat[k], test_DA[k]).flatten()

        loss=loss_funktion(output,test_labels[k])
        loss_list_test.append(loss.item())
 

        acc_list_test.append(np.sum((torch.round(torch.sigmoid(output)) == test_labels[k]).detach().numpy()))
            
        
    print(i,"Train Loss: %.2f Train Accuracy: %.2f Test Loss: %.2f Test Accuracy: %.2f"
        % (np.mean(loss_list_train), np.mean(acc_list_train),np.mean(loss_list_test), np.mean(acc_list_test)))

Sie sehen, das Training dauert sehr lange und ist nicht besonders erfolgreich. Das hie vorgestellte Models ist ein sehr, sehr einfaches. Tatsächlich werden normalerweise komplexere Graph Convolution genommen. Auch wird nicht nur der Atomtyp, sondern eine Vielzahl als Input für die Featurematrix genommen. 

Seit einigen Jahren gibt es auch [PyTorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/) und [Deep Graph Library](https://www.dgl.ai/). Beide stellen eine Erweiterung zu PyTorch. Sie enthalten wichtige Funktionalitäten, die relevant für den Umgang mit Graphen sind. Auch enthalten die Libraries, die wichtigsten Graph Layers. Dadurch müssen Sie die Layers sind nicht mehr selber schreiben.

## Übungsaufgabe:

Hier sehen Sie unser Graph Neural Network.
Das Problem ist, dass dieses Netzwerk keine Flexibilität bietet.
Die Gewichtsmatrizen des Netzwerks werden immer die selber Größen haben.
Auch fehlt Dropout. Wir brauchen kein Batchnorm, da wir keine Minibatches benutzen.


```python
class GraphNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GraphConvolution(25, 100)
        self.conv2 = GraphConvolution(100, 100)
        self.lin = nn.Linear(100, 1)
        
    def aggregate(self, convoluted_graph): 
        return torch.mean(convoluted_graph, dim=0, keepdim=True)
        
    def forward(self, x, adj):
        x = self.conv1(x, adj)
        x = F.relu(x)
        x = self.conv2(x, adj)
        x = F.relu(x)
        x = self.aggregate(x)
        x = self.lin(x)
        return x
```

Können Sie das Netzwerk so umschreiben, dass wir flexibel den Input, als auch die Hiddengrößen?
Sie können testen, ob ihr Netzwerk noch funktioniert mit dem `beispiel_DA` und `beispie_feat`.

In [ ]:
beispiel_DA = test_DA[0]
beispiel_feat = test_feat[0]

In [ ]:
class GraphNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GraphConvolution(25, 100)
        self.conv2 = GraphConvolution(100, 100)
        self.lin = nn.Linear(100, 1)
        
    def aggregate(self, convoluted_graph): 
        return torch.mean(convoluted_graph, dim=0, keepdim=True)
        
    def forward(self, x, adj):
        x = self.conv1(x, adj)
        x = F.relu(x)
        x = self.conv2(x, adj)
        x = F.relu(x)
        x = self.aggregate(x)
        x = self.lin(x)
        return x